**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

In [1]:
import bigdl.nano.automl as automl
import bigdl.nano.automl.hpo as hpo
automl.hpo_config.enable_hpo_tf()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import RMSprop

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Prepare data
We will load MNIST as our data.

In [2]:
CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
img_x, img_y = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(-1, img_x, img_y,1).astype("float32") / 255
x_test = x_test.reshape(-1, img_x, img_y,1).astype("float32") / 255

# Step3: Set search space
We now create our custom model class. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [3]:
@hpo.tfmodel()
class MyModel(tf.keras.Model):

    def __init__(self, filters, kernel_size, strides, activation):
        super().__init__()
        self.conv1 = Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            activation=activation)
        self.pool1 = MaxPooling2D(pool_size=2)
        self.drop1 = Dropout(0.3)
        self.flat = Flatten()
        self.dense1 = Dense(256, activation='relu')
        self.drop3 = Dropout(0.5)
        self.dense2 = Dense(CLASSES, activation="softmax")

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat(x)
        x = self.dense1(x)
        x = self.drop3(x)
        x = self.dense2(x)
        return x
model = MyModel(
    filters=hpo.space.Categorical(32, 64),
    kernel_size=hpo.space.Categorical(2, 4),
    strides=hpo.space.Categorical(1, 2),
    activation=hpo.space.Categorical("relu", "linear")
)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(learning_rate=5e-4),
    metrics=["accuracy"],
)

2022-05-25 15:06:47.542293: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-25 15:06:47.543561: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step4: Run hyperparameter search
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize or minimize it.

In [4]:
%%time
model.search(
    n_trials=5,
    target_metric='val_accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-25 15:06:47,620] A new study created in memory with name: no-name-2f770d53-1b57-445f-bb72-2f6e69ade389


Starting a new tuning
Epoch 1/24
375/375 - 30s - loss: 0.5556 - accuracy: 0.8084 - val_loss: 0.3723 - val_accuracy: 0.8668 - 30s/epoch - 79ms/step
Epoch 2/24
375/375 - 29s - loss: 0.3765 - accuracy: 0.8658 - val_loss: 0.3297 - val_accuracy: 0.8813 - 29s/epoch - 76ms/step
Epoch 3/24
375/375 - 30s - loss: 0.3328 - accuracy: 0.8813 - val_loss: 0.3059 - val_accuracy: 0.8888 - 30s/epoch - 79ms/step
Epoch 4/24
375/375 - 29s - loss: 0.3033 - accuracy: 0.8895 - val_loss: 0.2921 - val_accuracy: 0.8949 - 29s/epoch - 77ms/step
Epoch 5/24
375/375 - 29s - loss: 0.2844 - accuracy: 0.8968 - val_loss: 0.2850 - val_accuracy: 0.8968 - 29s/epoch - 77ms/step
Epoch 6/24
375/375 - 29s - loss: 0.2684 - accuracy: 0.9017 - val_loss: 0.2764 - val_accuracy: 0.9009 - 29s/epoch - 77ms/step
Epoch 7/24
375/375 - 29s - loss: 0.2572 - accuracy: 0.9067 - val_loss: 0.2692 - val_accuracy: 0.9028 - 29s/epoch - 77ms/step
Epoch 8/24
375/375 - 29s - loss: 0.2432 - accuracy: 0.9122 - val_loss: 0.2627 - val_accuracy: 0.9046 - 

[I 2022-05-25 15:18:40,523] Trial 0 finished with value: 0.9153333306312561 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 1, 'strides▁choice': 0}. Best is trial 0 with value: 0.9153333306312561.


Epoch 1/24
375/375 - 11s - loss: 0.6753 - accuracy: 0.7632 - val_loss: 0.4510 - val_accuracy: 0.8367 - 11s/epoch - 29ms/step
Epoch 2/24
375/375 - 11s - loss: 0.4516 - accuracy: 0.8370 - val_loss: 0.3902 - val_accuracy: 0.8594 - 11s/epoch - 28ms/step
Epoch 3/24
375/375 - 9s - loss: 0.4034 - accuracy: 0.8531 - val_loss: 0.3589 - val_accuracy: 0.8702 - 9s/epoch - 23ms/step
Epoch 4/24
375/375 - 10s - loss: 0.3729 - accuracy: 0.8650 - val_loss: 0.3391 - val_accuracy: 0.8792 - 10s/epoch - 26ms/step
Epoch 5/24
375/375 - 10s - loss: 0.3549 - accuracy: 0.8720 - val_loss: 0.3294 - val_accuracy: 0.8802 - 10s/epoch - 25ms/step
Epoch 6/24
375/375 - 9s - loss: 0.3379 - accuracy: 0.8761 - val_loss: 0.3211 - val_accuracy: 0.8848 - 9s/epoch - 23ms/step
Epoch 7/24
375/375 - 11s - loss: 0.3247 - accuracy: 0.8814 - val_loss: 0.3185 - val_accuracy: 0.8802 - 11s/epoch - 29ms/step
Epoch 8/24
375/375 - 8s - loss: 0.3173 - accuracy: 0.8848 - val_loss: 0.3035 - val_accuracy: 0.8906 - 8s/epoch - 20ms/step
Epoch 

[I 2022-05-25 15:21:44,764] Trial 1 finished with value: 0.9020833373069763 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 1, 'strides▁choice': 1}. Best is trial 0 with value: 0.9153333306312561.


...

Epoch 1/24
375/375 - 27s - loss: 0.6197 - accuracy: 0.7802 - val_loss: 0.4010 - val_accuracy: 0.8576 - 27s/epoch - 71ms/step
Epoch 2/24
375/375 - 24s - loss: 0.4063 - accuracy: 0.8558 - val_loss: 0.3456 - val_accuracy: 0.8757 - 24s/epoch - 65ms/step
Epoch 3/24
375/375 - 24s - loss: 0.3518 - accuracy: 0.8751 - val_loss: 0.3137 - val_accuracy: 0.8878 - 24s/epoch - 65ms/step
Epoch 4/24
375/375 - 24s - loss: 0.3222 - accuracy: 0.8842 - val_loss: 0.3017 - val_accuracy: 0.8902 - 24s/epoch - 64ms/step
Epoch 5/24
375/375 - 26s - loss: 0.3017 - accuracy: 0.8929 - val_loss: 0.2837 - val_accuracy: 0.8974 - 26s/epoch - 69ms/step
Epoch 6/24
375/375 - 25s - loss: 0.2873 - accuracy: 0.8970 - val_loss: 0.2778 - val_accuracy: 0.8989 - 25s/epoch - 67ms/step
Epoch 7/24
375/375 - 24s - loss: 0.2751 - accuracy: 0.9014 - val_loss: 0.2818 - val_accuracy: 0.8975 - 24s/epoch - 65ms/step
Epoch 8/24
375/375 - 28s - loss: 0.2671 - accuracy: 0.9044 - val_loss: 0.2666 - val_accuracy: 0.9053 - 28s/epoch - 76ms/step


[I 2022-05-25 15:37:01,656] Trial 3 finished with value: 0.9181666374206543 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 3 with value: 0.9181666374206543.


Epoch 1/24
375/375 - 14s - loss: 0.7663 - accuracy: 0.7320 - val_loss: 0.4939 - val_accuracy: 0.8191 - 14s/epoch - 37ms/step
Epoch 2/24
375/375 - 12s - loss: 0.5171 - accuracy: 0.8131 - val_loss: 0.4195 - val_accuracy: 0.8478 - 12s/epoch - 32ms/step
Epoch 3/24
375/375 - 11s - loss: 0.4606 - accuracy: 0.8360 - val_loss: 0.3911 - val_accuracy: 0.8584 - 11s/epoch - 30ms/step
Epoch 4/24
375/375 - 13s - loss: 0.4285 - accuracy: 0.8466 - val_loss: 0.3670 - val_accuracy: 0.8668 - 13s/epoch - 34ms/step
Epoch 5/24
375/375 - 16s - loss: 0.4078 - accuracy: 0.8536 - val_loss: 0.3722 - val_accuracy: 0.8608 - 16s/epoch - 42ms/step
Epoch 6/24
375/375 - 14s - loss: 0.3934 - accuracy: 0.8563 - val_loss: 0.3452 - val_accuracy: 0.8744 - 14s/epoch - 37ms/step
Epoch 7/24
375/375 - 16s - loss: 0.3806 - accuracy: 0.8639 - val_loss: 0.3376 - val_accuracy: 0.8739 - 16s/epoch - 43ms/step
Epoch 8/24
375/375 - 10s - loss: 0.3704 - accuracy: 0.8666 - val_loss: 0.3290 - val_accuracy: 0.8803 - 10s/epoch - 27ms/step


[I 2022-05-25 15:41:02,484] Trial 4 finished with value: 0.8978333473205566 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 3 with value: 0.9181666374206543.


CPU times: user 1h 22min 43s, sys: 1h 4min 50s, total: 2h 27min 34s
Wall time: 34min 14s


# Step 5: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [5]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Epoch 1/24
375/375 [==============================] - 19s 49ms/step - loss: 0.6140 - accuracy: 0.7824 - val_loss: 0.4432 - val_accuracy: 0.8396
Epoch 2/24
375/375 [==============================] - 17s 46ms/step - loss: 0.4155 - accuracy: 0.8529 - val_loss: 0.3558 - val_accuracy: 0.8719
Epoch 3/24
375/375 [==============================] - 16s 41ms/step - loss: 0.3652 - accuracy: 0.8709 - val_loss: 0.3216 - val_accuracy: 0.8869
Epoch 4/24
375/375 [==============================] - 14s 38ms/step - loss: 0.3338 - accuracy: 0.8815 - val_loss: 0.3056 - val_accuracy: 0.8901
Epoch 5/24
375/375 [==============================] - 14s 38ms/step - loss: 0.3133 - accuracy: 0.8872 - val_loss: 0.2950 - val_accuracy: 0.8942
Epoch 6/24
375/375 [==============================] - 15s 39ms/step - loss: 0.2981 - accuracy: 0.8925 - val_loss: 0.3018 - val_accuracy: 0.8902
Epoch 7/24
375/375 [==============================] - 15s 40ms/step - loss: 0.2883 - accuracy: 0.8974 - val_loss: 0.2827 - val_accuracy:

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [6]:
print(model.summary())

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           multiple                  160       
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         multiple                  0         
                                                                 
 flatten_1 (Flatten)         multiple                  0         
                                                                 
 dense_2 (Dense)             multiple                  1384704   
                                                                 
 dropout_3 (Dropout)         multiple                  0         
                                                        